# 基于MindSpore构造平方损失函数 
本实验主要介绍使用MindSpore构造平方损失函数，利用自定义的数据集和线性模型验证。

# 1、实验目的
- 掌握平方损失函数的原理
- 掌握如何使用MindSpore构造平方损失函数
- 了解如何使用MindSpore进行模型的训练和验证

# 2、 平方损失函数原理介绍
平方损失函数(Square Loss)用于测量机器学习模型的输出与实际结果之间的距离，平方损失函数一般不用于分类问题。。假设$y_i$是真实值，$\hat{y}_i$是预测值，则平方损失函数$J(\theta)$的公式如下：
$$J(\theta)=\frac{1}{2}\sum^{n}_{i=0}(y_i-\hat{y}_i)^2$$


本实验中平方损失函数通过重写L1Loss实现，L1范数是指向量中各个元素绝对值之和，对于向量$x=[x_1,x_2,…,x_n ]^T$，其L1范数为：
$$‖x‖_1=|x_1|+|x_2|+⋯+|x_n|$$

# 3、 实验环境
在动手进行实践之前，需要注意以下几点：
* 确保实验环境正确安装，包括安装MindSpore。安装过程：首先登录[MindSpore官网安装页面](https://www.mindspore.cn/install)，根据安装指南下载安装包及查询相关文档。同时，官网环境安装也可以按下表说明找到对应环境搭建文档链接，根据环境搭建手册配置对应的实验环境。
* 推荐使用交互式的计算环境Jupyter Notebook，其交互性强，易于可视化，适合频繁修改的数据分析实验环境。
* 实验也可以在华为云一站式的AI开发平台ModelArts上完成。
* 推荐实验环境：MindSpore版本=MindSpore 2.0；Python环境=3.7


|  硬件平台 |  操作系统  | 软件环境 | 开发环境 | 环境搭建链接 |
| :-----:| :----: | :----: |:----:   |:----:   |
| CPU | Windows-x64 | MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.1节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| GPU CUDA 10.1|Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.2节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| Ascend 910  | Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第四章](./MindSpore环境搭建实验手册.docx)|

# 4、数据处理
## 4.1 数据准备
这是一个自定义的简单数据集，包括两类数据'xy'和'z'，'xy'由x和y两列数据组成，它们都服从[-1,1]的均匀分布，'z'由它们生成，公式如下：
$$z=a*x^2+b*y^3+c+noise$$
其中，a、b、c是自定义的超参数，noise是服从(0,0.03)的正态分布。
数据集大小为160，分为10个批次，每个批次增强10倍。数据的形式如下：

$$xy=[[[ 7.56207947e-03, -3.43424603e-02]],\\
 [[ 4.21533167e-01, -4.53344226e-01]],\\
 [[ 1.59027276e-03,  8.72432720e-05]],\\
 ...\\
 [[ 3.67112603e-04, -2.49217793e-01]],\\
 [[ 1.86624035e-01, -1.43198028e-01]],\\
 [[ 7.65149057e-01, -1.37545183e-01]]]$$
 
$$z=
[[ 4.91964149e+00],\\
 [ 4.45919466e+00],\\
 [ 5.02036238e+00],\\
 ...\\
 [ 4.26575947e+00],\\
 [ 4.87499523e+00],\\
 [ 6.14354038e+00]]$$

## 4.2 数据加载
这里进行自定义数据集的生成和增强操作，首先`get_data`函数根据公式获取数据，然后在`create_dataset`函数生成数据集，并进行数据增强和处理操作：
- 定义进行数据增强和处理所需要的一些参数。
- 根据参数，生成对应的数据增强操作。
- 对生成的数据集进行处理。
- 对处理好的数据进行样例展示。


In [2]:
import numpy as np
from mindspore import dataset as ds

# 获取数据
def get_data(num, a=2.0, b=3.0, c=5.0):
    
    for _ in range(num):
        # 均匀分布
        x = np.random.uniform(-1.0, 1.0)
        y = np.random.uniform(-1.0, 1.0)
        # 添加噪声
        noise = np.random.normal(0, 0.03)
        z = a * x ** 2 + b * y ** 3 + c + noise
        yield np.array([[x**2], [y**3]],dtype=np.float32).reshape(1,2), np.array([z]).astype(np.float32)

# 生成数据集并增强
def create_dataset(num_data, batch_size=16, repeat_size=1):
    # 生成数据集
    input_data = ds.GeneratorDataset(list(get_data(num_data)), column_names=['xy','z'])
    # 划分批次
    input_data = input_data.batch(batch_size)
    # 增强数据集
    input_data = input_data.repeat(repeat_size)
    return input_data
 
data_number = 160       # 数据集大小
batch_number = 10       # 批量大小  
repeat_number = 10      # 增强次数

# 训练集
ds_train = create_dataset(data_number, batch_size=batch_number, repeat_size=repeat_number)
# 测试集
ds_test = create_dataset(data_number, batch_size=batch_number, repeat_size=repeat_number)

[WARNING] ME(7328:17100,MainProcess):2023-02-25-02:32:53.608.675 [mindspore\dataset\engine\datasets_user_defined.py:657] Python multiprocessing is not supported on Windows platform.


#  5、模型构建

模型的构建包括如下部分：
- 导入库和函数
- 定义线性模型
- 重写损失函数



## 导入所需库和函数

首先导入需要的Python库。

然后导入相关的MindSpore模块，详细的MindSpore的模块说明，可以在MindSpore API页面中搜索查询。

可以通过context.set_context来配置运行需要的信息，譬如运行模式、后端信息、硬件等信息。导入context模块，配置运行需要的信息。


In [3]:
# 时间处理模块
import time
# 科学计算库
import numpy as np
# MindSpore库
import mindspore as ms
# 常见算子操作
import mindspore.ops as ops
# 数据集处理模块
from mindspore import dataset as ds
# 神经网络模块，张量
from mindspore import nn, Tensor
# 模型训练设置
from mindspore.train import Callback, LossMonitor, Model
# L1型损失函数
from mindspore.nn import L1Loss
# MindSpore环境设置的0号种子
ms.set_seed(0)

## 定义一个简单的线性模型

这里是线性网络的最简单实现，一个全连接层，公式如下：
$$outputs=activation(X*kernel+bias)$$
其中 X 是输入Tensor， activation 是激活函数， kernel 是一个权重矩阵，其数据类型与 X 相同， bias 是一个偏置向量，其数据类型与 X 相同（仅当has_bias为True时）。

这里使用的是一个全连接层`mindspore.nn.Dense`，输入维度为2，输出维度为1，初始化权重为0.02，初始化偏置为0.02。

In [4]:
# 定义线性模型
class LinearNet(nn.Cell):
    
    def __init__(self):
        super(LinearNet, self).__init__()
        # 全连接层
        self.fc = nn.Dense(2, 1, 0.02, 0.02)
 
    def construct(self, x):
        # 前向传播
        x = self.fc(x)
        return x

查看模型中参数维度

In [5]:
start_time = time.time()
net = LinearNet()
model_params = net.trainable_params()

# 显示模型的参数及其大小
print ('Param Shape is: {}'.format(len(model_params)))
for net_param in net.trainable_params():
    print(net_param, net_param.asnumpy())

Param Shape is: 2
Parameter (name=fc.weight, shape=(1, 2), dtype=Float32, requires_grad=True) [[0.02 0.02]]
Parameter (name=fc.bias, shape=(1,), dtype=Float32, requires_grad=True) [0.02]


## 重写L1型损失函数，实现自定义的平方损失函数

`L1Loss`用于计算预测值和目标值之间的平均绝对误差。而平方损失函数只需要对L1Loss进行平方操作，并乘上0.5。

In [6]:
#自定义平方损失函数
class quadratic_loss(L1Loss):
    def __init__(self, reduction="mean"):
        super(quadratic_loss, self).__init__(reduction)
 
    def construct(self, base, target):
        # 平方损失函数
        x = 0.5 * ops.square(base - target)
        return self.get_loss(x)

user_loss = quadratic_loss()

# 6、模型训练

完成数据预处理、网络定义、损失函数之后，选择优化器，开始模型训练。模型训练包含1层迭代，数据集按分组从训练集中抽取数据，输入网络计算得到损失函数。

因为本实验只是计算平方损失函数值，不涉及梯度计算和反向传播，所以优化器的选择并不重要。


In [7]:
# 选择动量优化器
optim = nn.Momentum(net.trainable_params(), learning_rate=0.01, momentum=0.6)
# 使用Model接口将网络、损失函数和优化器关联起来
model = Model(net, user_loss, optim)
 
# 开始训练
epoch = 1
model.train(epoch, ds_train, callbacks=[LossMonitor(10)], dataset_sink_mode=True)
 
# 显示模型参数
for net_param in net.trainable_params():
    print(net_param, net_param.asnumpy())

# 显示训练时间
print ('The total time cost is: {}s'.format(time.time() - start_time))

epoch: 1 step: 10, loss is 14.196735382080078
epoch: 1 step: 20, loss is 6.716091156005859
epoch: 1 step: 30, loss is 4.461669445037842
epoch: 1 step: 40, loss is 2.9822726249694824
epoch: 1 step: 50, loss is 1.8455755710601807
epoch: 1 step: 60, loss is 1.0462146997451782
epoch: 1 step: 70, loss is 0.7596532702445984
epoch: 1 step: 80, loss is 1.0583897829055786
epoch: 1 step: 90, loss is 1.3813812732696533
epoch: 1 step: 100, loss is 0.9386886358261108
epoch: 1 step: 110, loss is 0.7798466682434082
epoch: 1 step: 120, loss is 0.9109617471694946
epoch: 1 step: 130, loss is 1.8585299253463745
epoch: 1 step: 140, loss is 0.8565757870674133
epoch: 1 step: 150, loss is 1.2890172004699707
epoch: 1 step: 160, loss is 0.4561425745487213
Parameter (name=fc.weight, shape=(1, 2), dtype=Float32, requires_grad=True) [[1.4566594  0.19539458]]
Parameter (name=fc.bias, shape=(1,), dtype=Float32, requires_grad=True) [5.1525297]
The total time cost is: 41.54380178451538s


# 7、模型预测

模型训练完成后，输入测试集进行模型预测，计算平方损失函数值。

模型预测时不需要设置优化器，只需要设置网络、损失函数和评估指标。

In [ ]:
# 模型预测
model = Model(net, loss_fn=user_loss, optimizer=None, metrics={'loss'})
# 计算测试集的平方损失函数值
pred_loss = model.eval(ds_test, dataset_sink_mode=False)
print(f'prediction loss is {pred_loss}')